In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/CS4375-Project/

/content/drive/MyDrive/CS4375-Project


In [ ]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipa
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
audio_dataset_path = '/content/drive/MyDrive/CS4375-Project/genres_original'
metadata = pd.read_csv('/content/drive/MyDrive/CS4375-Project/features_30_sec.csv')
metadata.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [ ]:
def features_extractor(file):
  audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
  mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
  mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

  return mfccs_scaled_features

In [ ]:
from tqdm import tqdm
extracted_features = []

for index_num, row in tqdm(metadata.iterrows()):
    try:
        final_class_labels = row["label"]
        file_name = os.path.join(os.path.abspath(audio_dataset_path), final_class_labels+'/', str(row["filename"]))
        data = features_extractor(file_name)
        extracted_features.append([data, final_class_labels])
    except Exception as e:
        print(f"Error: {e}")
        continue


553it [01:03, 11.45it/s]<ipython-input-63-2d99a3da4535>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
557it [01:05,  5.94it/s]

Error: 


1000it [01:47,  9.34it/s]


In [ ]:
extracted_features_df = pd.DataFrame(extracted_features,columns=['feature', 'class'])
extracted_features_df.head()

,feature,class
0,"[-113.59882, 121.57067, -19.162262, 42.36394, ...",blues
1,"[-207.52383, 123.98514, 8.947019, 35.86715, 2....",blues
2,"[-90.757164, 140.44087, -29.084547, 31.686693,...",blues
3,"[-199.57513, 150.0861, 5.663404, 26.855282, 1....",blues
4,"[-160.35417, 126.20948, -35.581394, 22.139256,...",blues


In [ ]:
X = np.array(extracted_features_df['feature'].tolist())
y = np.array(extracted_features_df['class'].tolist())

In [ ]:
X.shape

(999, 40)

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))

In [ ]:
y.shape

(999, 10)

Split Training and test data

In [ ]:
# create train, validation and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2)

# add an axis to input sets
X_train = X_train[..., np.newaxis]
X_validation = X_validation[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)

CNN Model Implementation:

In [ ]:
# Building network topology
model = keras.Sequential()

# 1st conv layer
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(keras.layers.MaxPooling2D((2, 2), padding='same'))
model.add(keras.layers.BatchNormalization())

# 2nd conv layer
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D((2, 2), padding='same'))
model.add(keras.layers.BatchNormalization())

# 3rd conv layer
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D((2, 2), padding='same'))
model.add(keras.layers.BatchNormalization())

# flatten output and feed it into dense layer
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))

# Output layer
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

Compiling model

In [ ]:
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 40, 1, 32)         320       
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 20, 1, 32)         0         
 ng2D)                                                           
                                                                 
 batch_normalization_18 (Ba  (None, 20, 1, 32)         128       
 tchNormalization)                                               
                                                                 
 conv2d_22 (Conv2D)          (None, 20, 1, 64)         18496     
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 10, 1, 64)         0         
 ng2D)                                                           
                                                      

Training Model

In [ ]:
# train model
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=75)

Epoch 1/75
19/19 [==============================] - 2s 31ms/step - loss: 2.1159 - accuracy: 0.2972 - val_loss: 2.2404 - val_accuracy: 0.1133
Epoch 2/75
19/19 [==============================] - 0s 19ms/step - loss: 1.6223 - accuracy: 0.4341 - val_loss: 2.1519 - val_accuracy: 0.1667
Epoch 3/75
19/19 [==============================] - 0s 19ms/step - loss: 1.4011 - accuracy: 0.5192 - val_loss: 2.0356 - val_accuracy: 0.2600
Epoch 4/75
19/19 [==============================] - 0s 19ms/step - loss: 1.2565 - accuracy: 0.5710 - val_loss: 1.9717 - val_accuracy: 0.3067
Epoch 5/75
19/19 [==============================] - 0s 20ms/step - loss: 1.1927 - accuracy: 0.5927 - val_loss: 1.8618 - val_accuracy: 0.4800
Epoch 6/75
19/19 [==============================] - 0s 17ms/step - loss: 1.1749 - accuracy: 0.5776 - val_loss: 1.8244 - val_accuracy: 0.4600
Epoch 7/75
19/19 [==============================] - 0s 17ms/step - loss: 1.0742 - accuracy: 0.6227 - val_loss: 1.7631 - val_accuracy: 0.4733
Epoch 8/75
19

Model Evaluation

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)

# Print the test loss and accuracy
print("\nTest Accuracy:", accuracy)
print("Test Loss:", loss)

8/8 - 0s - loss: 1.7531 - accuracy: 0.6720 - 113ms/epoch - 14ms/step

Test Accuracy: 0.671999990940094
Test Loss: 1.7530802488327026
